# Summary of `xyBnG` results

The summaries are stored in two files. One is ``scenario.par``This is a text file about the scenario parameters. The other is a serialized `DataFrame`, called `summary.ser`. Below are example codes to retrieve the records.

In [ ]:
using DataFrames, Serialization, Statistics, Plots, Dates

In [ ]:
dir = "."  # modify this to your work directory
pars = readlines("$dir/scenario.par")
start = Dates.unix2datetime(parse(Float64, split(pars[1])[2]))
println("Simulation started: $start")
stop, mature = begin
    if split(pars[end])[1] == "Ended:"
        Dates.unix2datetime(parse(Float64, split(pars[end])[2])), true
    else
        println("Simulation was not finised.")
        Dates.unix2datetime(mtime("$dir/summary.ser")), false
    end
end
println("Simulation finised: $stop")
println()
if mature
    print(join(pars[2:end-1], '\n'))
else
    print(join(pars[2:end], '\n'))
end

In [ ]:
rst = deserialize("$dir/summary.ser")
describe(rst)

In [ ]:
# What schemes used in the simulatino
schemes = unique(rst.scheme)

In [ ]:
dic = Dict{String,DataFrame}()
mmm = Dict{String,DataFrame}()
for scheme in schemes
    dic[scheme] = select(filter(x -> x.scheme == scheme, rst), Not(:scheme))
    mmm[scheme] = combine(groupby(dic[scheme], :grt), Not(:repeat) .=> mean .=> Not(:repeat))
    mmm[scheme][!, :ceiling] *= 2
    mmm[scheme][!, :floor] *= 2
end
describe(mmm["aaocs"])

In [ ]:
function sumplot(ddf, ss, lbl, par...; pos=:best)
    c = 1
    ls = (:solid, :dash, :dashdot, :dashdotdot, :dot)
    p = plot(xlabel="Generation", ylabel=lbl, legend=pos)
    for s in ss
        df = ddf[s]
        l = 1
        for y in par
            plot!(p, df[!, :grt] .- 5, df[!, y], label=s * '-' * y, color=c, linestyle=ls[l])
            l += 1
            l > 5 && (l = 1)
        end
        c += 1
    end
    p
end

In [ ]:
ss = ("ggocs", "iiocs", "agocs", "gblup")

In [ ]:
sumplot(mmm, ss, "N parents", "nsire", "ndam")

In [ ]:
sumplot(mmm, ss, "TBV", "mtbv")

In [ ]:
sumplot(mmm, ss, "Boundaries", "floor", "ceiling"; pos=:left)

In [ ]:
sumplot(mmm, ["ggocs"], "Inbreeding", "fibd", "fped", "fhet", "fdrift")

In [ ]:
sumplot(mmm, ss, "Var(TBV)", "vtbv", "genicv")

In [ ]:
sumplot(mmm, ss, "QTL lost", "xfq", "xuq")

In [ ]:
sumplot(mmm, ss, "Loci fixed", "xref", "xqtl")

In [ ]:
sumplot(mmm, ss, "Δq vs q0", "covdq", "covdq2")